# 模拟运行环境简介

AlphaMed 平台是以支持联邦学习任务为核心的平台，因此 AlphaMed 平台上的任务通常是需要很多参与方共同完成的。但是在任务的开发、测试阶段，这个实际的运行条件往往是不能满足的。一方面很多参与方可能还不具备条件实际加入进来；另一方面初期的代码需要很多调试修改的工作，本地调试运行无疑更高效、更便捷。AlphaMed 模拟运行环境正是为了解决本地调试联邦代码问题而提供的方案。

## 模拟运行环境运行机制

模拟运行环境是一个全局性环境，与具体的开发任务无关，AlphaMed 平台提供的所有基础组件均支持运行在模拟环境下。但是限于模拟行为本身的局限性，不同 AlphaMed 平台组件在模拟环境中的执行结果与在实际运行环境中的执行结果，可能一致也可能存在少许差异。差异主要体现在两个方面：
1. 只能访问本地资源，不会实际触碰任何外部资源。比如所有的参与方任务，实际都在本地环境运行，访问的都是本地的软硬件资源。
2. 当前仅模拟计算和通信行为，还不能模拟项目管理、任务管理、权限管理等高级管理功能。

在模拟运行环境中运行代码的方式非常简单，只需要将需要执行的代码放在 `mock_context` 上下文管理器中即可。

In [ ]:
from alphafed import mock_context

with mock_context():
    # 模拟执行代码
    ...

在进入 `mock_context` 管理器时，平台会自动搭建模拟运行环境，接着后续代码都会在模拟环境中运行。在模拟环境中，一部分底层组件的运行结果会发生变化，比如 `Scheduler` 的 `launch_task` 接口不会实际触发平台启动联邦学习任务。这是因为 AlphaMed 平台是一个隐私安全的平台，在实际用户自身确认参与一个指定计算任务前，任何其它用户都无权向其发送消息（除邀请请求），更不可能在其运行环境中执行任何操作。所以在模拟模式中，所有的代码都执行在本地、仅限于操纵本地资源。当模拟代码执行完毕后退出 `mock_context` 管理器时，平台会拆除模拟环境，之后的代码重新运行在常规 Python 环境中。

当前版本模拟环境主要处理三类任务：
1. 与外部接口的交互，比如调用任务管理器接口；
> 这一类任务中的大部分情况只是提供一个空实现，并不会实际触发平台任务数据发生任何变化。剩下的一部分接口可以通过人工配置模拟接口行为，比如 `mock_context` 的 `nodes` 参数可以设置当前模拟任务的参与方 ID 列表。还有一小部分可以直接使用接口功能，比如文件上传。
2. 合约消息的发送、接收；
> 这一类任务会在本地管理合约消息分发、模拟实际接口的行为，从发送、接收消息的结果上看与实际调用合约接口时完全一致。
3. 数据传输。
> 这一类任务会实际模拟数据的发送与接收，从结果上看与实际调用接口时完全一致，但数据不会被长期保存，即使设置了长期保存的参数。

## 模拟运行环境工具接口简介

以下为模拟运行环境接口的说明：

In [ ]:
from typing import List

In [ ]:
class mock_context:

    def __init__(self,
                 clean: bool = False,
                 id: str = None,
                 nodes: List[str] = None) -> None:
        """mock_context 实际上是一个上下文管理器，严格的讲不是接口。实际使用时配合 with 语句工作。

        参数说明:
            clean: 
                是否清空模拟运行环境的数据。一般情况下以前运行过程中产生的数据会保留，比如以前发送
                过的合约消息，这样 异常中断后下次可以继续在之前的环境中继续运行。但是有时候也会希望
                从头开始运行，此时残留数据可能会影响 代码正常执行，可以使用 clean 参数清空以前的
                数据。
                默认不清空数据。
            id:
                设置当前模拟节点的 ID。涉及多方配合的流程需要为每一个参与方指定唯一的 ID，模拟环境
                中不存在实际的参与方，所以通过此参数配置模拟 ID。
                默认使用随机生成的 UUID。
            nodes:
                参与当前任务的所有节点 ID 列表，多方配合的任务场景中经常会用到。实际运行时由任务
                管理器提供，模拟环境里任务还没有实际创建，所以通过 nodes 参数配置。
                默认设置为空列表。
        """

In [ ]:
# 使用示例
from alphafed import mock_context

self_node = 'NODE_ID_1'
partner_a = 'NODE_ID_2'
partner_b = 'NODE_ID_3'

with mock_context(id=self_node, nodes=[self_node, partner_a, partner_b], clean=True):
    # 模拟执行代码
    ...

In [1]:
def is_mock_mode():
    """返回当前代码位置是否运行于模拟环境中。"""

# 使用示例
from alphafed import mock_context, is_mock_mode

print(f'1. 当前是否运行于模拟环境中? {is_mock_mode()}')
with mock_context():
    print(f'2. 当前是否运行于模拟环境中? {is_mock_mode()}')
print(f'3. 当前是否运行于模拟环境中? {is_mock_mode()}')

2023-02-03 02:37:47,879|WARNING|mock|__enter__|89:
Without specifying nodes, query_nodes returns an empty list.


1. 当前是否运行于模拟环境中? False
2. 当前是否运行于模拟环境中? True
3. 当前是否运行于模拟环境中? False


In [3]:
def mock_node_id():
    """返回当前模拟节点 ID，仅在模拟环境中时有效。"""

# 使用示例
from alphafed import mock_context, mock_node_id

with mock_context():
    print('默认随机生成节点 ID')
    print(f'当前节点 ID 为: {mock_node_id()}')
with mock_context(id='849309c9-b0ae-4b87-8812-3ae970499fe9'):
    print('也可以指定节点 ID')
    print(f'当前节点 ID 为: {mock_node_id()}')

2023-02-03 02:39:24,651|WARNING|mock|__enter__|89:
Without specifying nodes, query_nodes returns an empty list.
2023-02-03 02:39:24,658|WARNING|mock|__enter__|89:
Without specifying nodes, query_nodes returns an empty list.


默认随机生成节点 ID
当前节点 ID 为: 46c54bb5-d1fa-48db-9f94-5e72eae9e3f6
也可以指定节点 ID
当前节点 ID 为: 849309c9-b0ae-4b87-8812-3ae970499fe9


In [4]:
def mock_nodes():
    """返回当前模拟任务的所有参与方节点 ID，仅在模拟环境中时有效。"""

# 使用示例
from alphafed import mock_context, mock_nodes

with mock_context(nodes=['node 1', 'node 2', 'node 3']):
    print(f'当前任务的参与方有: {mock_nodes()}')

当前任务的参与方有: ['node 1', 'node 2', 'node 3']


在介绍[横向联邦](3.%20%E6%A8%AA%E5%90%91%E8%81%94%E9%82%A6%E5%AD%A6%E4%B9%A0%E7%8E%AF%E5%A2%83%E7%AE%80%E4%BB%8B.ipynb)、[异构联邦](4.%20%E5%BC%82%E6%9E%84%E8%81%94%E9%82%A6%E5%AD%A6%E4%B9%A0%E7%8E%AF%E5%A2%83%E7%AE%80%E4%BB%8B.ipynb)时都已经实际使用模拟环境调试运行过计算任务，读者可参考使用。